#Alexa Schlotter and Kangzhe Zhou
##Final Project: Amino Acid Evolution in COVID-19 Variants (2021-2022): Correlating Changes with Case Dynamics, Hospitalizations, and Testing Efficacy in New York City

##Install Prerequisites

In [ ]:
!pip install biopython
!pip install ipytree
!pip install scikit-allel
!pip install zarr
!pip install pysam
!pip install pyvcf
!pip install numcodecs
!pip install dask
!wget https://mafft.cbrc.jp/alignment/software/mafft_7.471-1_amd64.deb
!sudo dpkg -i mafft_7.471-1_amd64.deb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 63.8 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5033 sha256=a31009ff4111e3a0dcb8453ce6fd887fe0395c08c5724803e220ed668cb5313d
  Stored in directory: /root/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built asciitree
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 63.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subproce

##Mount Google Drive

In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount("/content/drive")
DATA = Path("/content/drive/My Drive/Fall 2023/Introduction to Genomic Information Science and Technology/ECBME4060 Final Project/Variant FASTQ Files/DATA")


Mounted at /content/drive


##Sequence Preprocessing and MAFFT

In [ ]:
from Bio.Align.Applications import MafftCommandline
from Bio import SeqIO, AlignIO
import os, subprocess
import gzip
import shutil

!apt-get install -y seqtk
!sudo apt-get install -y clustalw
!sudo apt-get install -y clustalo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  seqtk
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 30.2 kB of archives.
After this operation, 85.0 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 seqtk amd64 1.3-2 [30.2 kB]
Fetched 30.2 kB in 0s (282 kB/s)
Selecting previously unselected package seqtk.
(Reading database ... 120978 files and directories currently installed.)
Preparing to unpack .../archives/seqtk_1.3-2_amd64.deb ...
Unpacking seqtk (1.3-2) ...
Setting up seqtk (1.3-2) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  clustalx seaview
The following NEW packages will be installed:
  clustalw
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 275 kB of a

In [ ]:
def run_command(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        raise Exception(f"Error with command '{command}': {stderr.decode('utf-8')}")
    return stdout.decode('utf-8')

def filter_sequences(input_file, output_file, max_n_percentage=0):
    sequences_kept = 0
    sequences_filtered = 0

    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for record in SeqIO.parse(infile, "fasta"):
            sequence_length = len(record.seq)
            n_count = record.seq.upper().count('N')
            n_percentage = (n_count / sequence_length) * 100

            if n_percentage <= max_n_percentage:
                SeqIO.write(record, outfile, "fasta")
                sequences_kept += 1
            else:
                sequences_filtered += 1

    return sequences_kept, sequences_filtered

def run_mafft_alignment(fasta_path, aligned_path):
    try:
        mafft_cline = MafftCommandline(input=fasta_path,thread = 1)
        stdout, stderr = mafft_cline()
        with open(aligned_path, "w") as aligned_file:
            aligned_file.write(stdout)
        print(f"Aligned {os.path.basename(fasta_path)}")
    except Exception as e:
        print(f"An error occurred while aligning {os.path.basename(fasta_path)}: {e}")

def select_random_sequences(input_file, output_file, num_sequences):
    all_sequences = list(SeqIO.parse(input_file, "fasta"))
    selected_sequences = random.sample(all_sequences, min(num_sequences, len(all_sequences)))

    with open(output_file, "w") as output_handle:
        SeqIO.write(selected_sequences, output_handle, "fasta")


Convert FASTQ files to FASTA files

In [ ]:
for filename in os.listdir(DATA):
    if filename.endswith('.fastq'):
        fastq_path = os.path.join(DATA, filename)
        fasta_path = fastq_path.replace('.fastq', '.fasta')

        # Convert FASTQ to FASTA
        try:
            run_command(f"seqtk seq -a \"{fastq_path}\" > \"{fasta_path}\"")
            print(f"Converted {filename} to FASTA format.")

        except Exception as e:
            print(f"An error occurred: {e}")

Converted 6_21_2_SRR18583056.fastq to FASTA format.
Converted 4_21_1_SRR16241829.fastq to FASTA format.
Converted 11_21_1_SRR18583067.fastq to FASTA format.
Converted 7_21_1_SRR18583053.fastq to FASTA format.
Converted 12_21_1_SRR18583064.fastq to FASTA format.
Converted 8_21_2_SRR18583059.fastq to FASTA format.
Converted 6_21_1_SRR18583055.fastq to FASTA format.
Converted 8_21_1_SRR18583054.fastq to FASTA format.
Converted 11_21_2_SRR18583069.fastq to FASTA format.
Converted 5_21_1_SRR15282875.fastq to FASTA format.
Converted 12_21_2_SRR18583065.fastq to FASTA format.
Converted 9_21_1_SRR18583058.fastq to FASTA format.
Converted 4_21_2_SRR16241828.fastq to FASTA format.
Converted 3_21_1_SRR18583085.fastq to FASTA format.
Converted 7_21_2_SRR18583157.fastq to FASTA format.
Converted 10_21_2_SRR18583259.fastq to FASTA format.
Converted 1_21_2_SRR14010094.fastq to FASTA format.
Converted 9_21_2_SRR18583088.fastq to FASTA format.
Converted 5_21_2_SRR15282876.fastq to FASTA format.
Convert

Filter out sequences that have missing nucleotides

In [ ]:
for filename in os.listdir(DATA):
    if filename.endswith('.fasta'):
        input_fasta = os.path.join(DATA, filename)
        output_fasta = os.path.join(DATA, filename.replace('.fasta', '_filtered.fasta'))
        kept, filtered = filter_sequences(input_fasta, output_fasta)
        print(f"File: {filename}, Sequences kept: {kept}, Sequences filtered out: {filtered}")

File: 6_21_2_SRR18583056.fasta, Sequences kept: 215221, Sequences filtered out: 3
File: 4_21_1_SRR16241829.fasta, Sequences kept: 255291, Sequences filtered out: 5
File: 11_21_1_SRR18583067.fasta, Sequences kept: 235138, Sequences filtered out: 14
File: 7_21_1_SRR18583053.fasta, Sequences kept: 277816, Sequences filtered out: 8
File: 12_21_1_SRR18583064.fasta, Sequences kept: 288934, Sequences filtered out: 274
File: 8_21_2_SRR18583059.fasta, Sequences kept: 270071, Sequences filtered out: 7725
File: 6_21_1_SRR18583055.fasta, Sequences kept: 254782, Sequences filtered out: 12
File: 8_21_1_SRR18583054.fasta, Sequences kept: 316817, Sequences filtered out: 13
File: 11_21_2_SRR18583069.fasta, Sequences kept: 339462, Sequences filtered out: 280
File: 5_21_1_SRR15282875.fasta, Sequences kept: 325583, Sequences filtered out: 17
File: 12_21_2_SRR18583065.fasta, Sequences kept: 304183, Sequences filtered out: 1809
File: 9_21_1_SRR18583058.fasta, Sequences kept: 312069, Sequences filtered out: 

Select 2000 random sequences from each file to allow for computation to stay within limits of system

In [ ]:
from Bio import SeqIO
import random

num_to_select = 2000

for filename in os.listdir(DATA):
    if filename.endswith('_filtered.fasta'):
        input_path = os.path.join(DATA, filename)
        output_path = os.path.join(DATA, f"subset_{filename}")
        select_random_sequences(input_path, output_path, num_to_select)
        print(f"Processed subset for {filename}")

Processed subset for 6_21_2_SRR18583056_filtered.fasta
Processed subset for 4_21_1_SRR16241829_filtered.fasta
Processed subset for 11_21_1_SRR18583067_filtered.fasta
Processed subset for 7_21_1_SRR18583053_filtered.fasta
Processed subset for 12_21_1_SRR18583064_filtered.fasta
Processed subset for 8_21_2_SRR18583059_filtered.fasta
Processed subset for 6_21_1_SRR18583055_filtered.fasta
Processed subset for 8_21_1_SRR18583054_filtered.fasta
Processed subset for 11_21_2_SRR18583069_filtered.fasta
Processed subset for 5_21_1_SRR15282875_filtered.fasta
Processed subset for 12_21_2_SRR18583065_filtered.fasta
Processed subset for 9_21_1_SRR18583058_filtered.fasta
Processed subset for 4_21_2_SRR16241828_filtered.fasta
Processed subset for 3_21_1_SRR18583085_filtered.fasta
Processed subset for 7_21_2_SRR18583157_filtered.fasta
Processed subset for 10_21_2_SRR18583259_filtered.fasta
Processed subset for 1_21_2_SRR14010094_filtered.fasta
Processed subset for 9_21_2_SRR18583088_filtered.fasta
Proce

## Mutation calling

First, we need to combine the sequences with the reference sequence and align the sequences to the reference sequence.

In [ ]:
def align_with_mafft(reference_file, target_file, aligned_file):

    combined_file = 'combined.fasta'
    with open(combined_file, 'w') as combined:
        with open(reference_file, 'r') as ref_fasta, open(target_file, 'r') as target_fasta:
            combined.write(ref_fasta.read() + '\n' + target_fasta.read())


    mafft_cline = MafftCommandline(input=combined_file)
    stdout, stderr = subprocess.Popen(str(mafft_cline),
                                      stdout=subprocess.PIPE,
                                      stderr=subprocess.PIPE,
                                      shell=True).communicate()

    with open(aligned_file, 'w') as aligned:
        aligned.write(stdout.decode())

    os.remove(combined_file)

DATA_aligned = DATA

reference_path = DATA_aligned / "MN908947.3_spike_protein.fasta"


for filename in os.listdir(DATA_aligned):
    if filename.endswith('.fasta') and filename.startswith('subset_'):
        sequence_path = DATA_aligned / filename
        aligned_sequence_path = DATA_aligned / f"ref_aligned_{filename}"

        # Align the sequences
        align_with_mafft(reference_path, sequence_path, aligned_sequence_path)

        # Generate mutation report on aligned sequences
        report_path = DATA / f"mutations_{filename}.txt"
        #call_mutations(reference_path, aligned_sequence_path, report_path)
        print(f"Aligned to reference {filename}")

Aligned to reference subset_6_21_2_SRR18583056_filtered.fasta
Aligned to reference subset_4_21_1_SRR16241829_filtered.fasta
Aligned to reference subset_11_21_1_SRR18583067_filtered.fasta
Aligned to reference subset_7_21_1_SRR18583053_filtered.fasta
Aligned to reference subset_12_21_1_SRR18583064_filtered.fasta
Aligned to reference subset_8_21_2_SRR18583059_filtered.fasta
Aligned to reference subset_6_21_1_SRR18583055_filtered.fasta
Aligned to reference subset_8_21_1_SRR18583054_filtered.fasta
Aligned to reference subset_11_21_2_SRR18583069_filtered.fasta
Aligned to reference subset_5_21_1_SRR15282875_filtered.fasta
Aligned to reference subset_12_21_2_SRR18583065_filtered.fasta
Aligned to reference subset_9_21_1_SRR18583058_filtered.fasta
Aligned to reference subset_4_21_2_SRR16241828_filtered.fasta
Aligned to reference subset_3_21_1_SRR18583085_filtered.fasta
Aligned to reference subset_7_21_2_SRR18583157_filtered.fasta
Aligned to reference subset_10_21_2_SRR18583259_filtered.fasta
Ali

convert each codon to its corresponding amino acid, accounting for the shift in reading window identified by NCBI.

In [ ]:
import os
from Bio import SeqIO
from Bio.Data import CodonTable

def translate_codons(seq, table):
    amino_acids = []
    for i in range(0, len(seq), 3):
        codon = seq[i:i+3]
        if codon == '---':
            amino_acids.append('-')
        else:
            amino_acids.append(table.forward_table.get(codon, 'X'))
    return amino_acids

def find_mutations(ref_aa_seq, target_aa_seq):
    mutations = []
    for pos, (ref_aa, target_aa) in enumerate(zip(ref_aa_seq, target_aa_seq)):
        if ref_aa != target_aa:
            mutations.append(f"{ref_aa}{pos+341}{target_aa}") #the + 341 accounts for the positioning of the covid spike protein in the amino acid sequence
    return mutations

# Genetic code table
genetic_code_table = CodonTable.standard_dna_table
for filename in os.listdir(DATA):
    if filename.startswith('ref_'):
        file_path = os.path.join(DATA, filename)
        sequences = list(SeqIO.parse(file_path, "fasta"))

        # Translate the reference sequence into amino acids
        ref_seq = sequences[0]
        ref_aa_seq = translate_codons(str(ref_seq.seq).upper(), genetic_code_table)
        output_file_path = os.path.join(DATA, f"mutations_{filename}.txt")

        with open(output_file_path, 'w') as output_file:
            for record in sequences[1:]:
                target_aa_seq = translate_codons(str(record.seq).upper(), genetic_code_table)
                mutations = find_mutations(ref_aa_seq, target_aa_seq)

                output_file.write(f"{record.id} mutations: {', '.join(mutations)}\n")

        print(f"Mutation report written to {output_file_path}")

Mutation report written to /content/drive/My Drive/Fall 2023/Introduction to Genomic Information Science and Technology/ECBME4060 Final Project/Variant FASTQ Files/DATA/mutations_ref_aligned_subset_6_21_2_SRR18583056_filtered.fasta.txt
Mutation report written to /content/drive/My Drive/Fall 2023/Introduction to Genomic Information Science and Technology/ECBME4060 Final Project/Variant FASTQ Files/DATA/mutations_ref_aligned_subset_4_21_1_SRR16241829_filtered.fasta.txt
Mutation report written to /content/drive/My Drive/Fall 2023/Introduction to Genomic Information Science and Technology/ECBME4060 Final Project/Variant FASTQ Files/DATA/mutations_ref_aligned_subset_11_21_1_SRR18583067_filtered.fasta.txt
Mutation report written to /content/drive/My Drive/Fall 2023/Introduction to Genomic Information Science and Technology/ECBME4060 Final Project/Variant FASTQ Files/DATA/mutations_ref_aligned_subset_7_21_1_SRR18583053_filtered.fasta.txt
Mutation report written to /content/drive/My Drive/Fall

## Variant Calling

Compare each set of mutations for a sequence with those of a variant, as demonstrated by literature: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9504725/

In [ ]:
import csv, collections

def read_variant_mutations(csv_file):
    variants = {}
    with open(csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            variant = row[0]
            mutations = set(filter(None, [mutation.upper() for mutation in row[1:]]))
            variants[variant] = mutations
    return variants

variants_csv = DATA / "variant_info.csv"
variants = read_variant_mutations(variants_csv)

variants

{'Alpha (B.1.1.7)': {'A570D',
  'D1118H',
  'D614G',
  'HV69-',
  'N501Y',
  'P681H',
  'S982A',
  'T716I',
  'Y144'},
 'Beta (B.1.351)': {'A701V',
  'D215G',
  'D614G',
  'D80A',
  'E484K',
  'K417N',
  'L18F',
  'LAL242-',
  'N501Y',
  'R246I'},
 'Gamma (P.1)': {'D614G',
  'E484K',
  'H655Y',
  'K417T',
  'N501Y',
  'T1027I',
  'V1176F'},
 'Zeta (P.2)': {'D614G', 'E484K', 'V1176F'},
 'Eta (B.1.525)': {'A67V',
  'D614G',
  'E484K',
  'F888L',
  'HV69-',
  'Q52R',
  'Q677H',
  'Y144'},
 'Kappa (B.1.617.1)': {'D614G',
  'E154K',
  'E484Q',
  'G142D',
  'L452R',
  'P681R',
  'Q1071H',
  'T95I'},
 'Delta (B.1.617)': {'D614G',
  'D950N',
  'FR157-158',
  'L452R',
  'P681R',
  'T19R',
  'T478K'},
 'Delta (AY.1)': {'D614G',
  'D950N',
  'FR157-158',
  'G142D',
  'K417N',
  'L452R',
  'P681R',
  'T19R',
  'T478K',
  'W258L'},
 'Delta (AY.2)': {'A222V',
  'D614G',
  'D950N',
  'FR157-158',
  'G142D',
  'K417N',
  'L452R',
  'P681R',
  'T19R',
  'T478K',
  'V70F'},
 'Delta (AY.4)': {'D614G',
  

In [ ]:
import os

def calculate_similarity(seq_mutations, var_mutations):
    seq_mut_set = set([mutation for mutation in seq_mutations.split(",")])
    var_mut_set = set([mutation for mutation in var_mutations])
    common_mutations = seq_mut_set.intersection(var_mut_set)
    similarity = len(common_mutations) / max(len(seq_mut_set), len(var_mut_set))
    return similarity, common_mutations

def find_closest_variant(seq_mutations, variants_dict):
    closest_variant = None
    max_similarity = 0
    overlapping_mutations = set()

    for variant, var_mutations in variants_dict.items():
        similarity, common_mutations = calculate_similarity(seq_mutations, var_mutations)
        if similarity > max_similarity:
            max_similarity = similarity
            closest_variant = variant
            overlapping_mutations = common_mutations

    return closest_variant, max_similarity, overlapping_mutations

for filename in os.listdir(DATA):
    if filename.startswith("mutations_ref_"):  # Modify the condition based on your file extension
        input_file_path = os.path.join(DATA, filename)
        output_file_path = os.path.join(DATA, f"{filename}_variants.txt")

        with open(output_file_path, "w") as output:
            for line in open(input_file_path):
                if "mutations:" in line:
                    sequence_name, mutations = line.strip().split(" mutations: ")
                    closest_variant, similarity, overlapping_mutations = find_closest_variant(mutations, variants)
                    output.write(f"{sequence_name} closest_variant: {closest_variant}, similarity: {similarity}\n")
                    output.write(f"Overlapping Mutations: {', '.join(set(variants[closest_variant]))}\n")

##Summarization

In [ ]:
import os
from collections import defaultdict

def process_file(input_file_path):
    variant_count = defaultdict(int)
    variant_mutations = defaultdict(set)

    with open(input_file_path, 'r') as file:
        for line in file:
            if 'closest_variant:' in line:
                parts = line.split('closest_variant: ')
                variant = parts[1].split(',')[0].strip()
                variant_count[variant] += 1

            if 'Overlapping Mutations:' in line:
                mutations = line.split('Overlapping Mutations: ')[1].strip().split(', ')
                variant_mutations[variant].update(mutations)

    return variant_count, variant_mutations

for filename in os.listdir(DATA):
    if filename.startswith("mutations_ref_") and filename.endswith("_variants.txt"):
        input_file_path = os.path.join(DATA, filename)
        variant_count, variant_mutations = process_file(input_file_path)

        summary_file_path = os.path.join(DATA, f"{filename}_summary.txt")
        with open(summary_file_path, 'w') as summary_file:
            for variant, count in variant_count.items():
                summary_file.write(f"Variant: {variant}, Sequence Count: {count}\n")
                mutations = ', '.join(variant_mutations[variant])
                summary_file.write(f"Associated Mutations: {mutations}\n\n")

print("Summary files created successfully.")

Summary files created successfully.
